 # Load Data

In [1]:
# azureml-core of version 1.0.72 or higher is required
# azureml-dataprep[pandas] of version 1.1.34 or higher is required
from azureml.core import Workspace, Dataset, Experiment

ws = Workspace.from_config()

experiment_name = "heart_predictions"
experiment = Experiment(ws, experiment_name)

dataset = Dataset.get_by_name(ws, name='heart')
dataset.to_pandas_dataframe()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3,0
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3,0
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3,0
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0


 # Create Compute

In [2]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "auto-ml"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 0, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


 # Hyperdrive Config

In [3]:
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import BayesianParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = BayesianParameterSampling(
        {
            "--penalty": uniform(0.01, 10),
            "--kernel": choice("linear", "poly", "rbf", "sigmoid")
        }
    )

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='./train/conda_env_v_1_0_0.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig("./train", 
    script="train.py", 
    compute_target=amlcompute_cluster_name, 
    environment=sklearn_env, 
    arguments=["--input-data", "heart"])

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
    hyperparameter_sampling=ps,
    primary_metric_name="Accuracy", 
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
    max_total_runs=40,
    max_concurrent_runs=4)

 # Train

In [19]:
run = experiment.submit(hyperdrive_config)

In [4]:
from azureml.train.hyperdrive.run import HyperDriveRun
run = HyperDriveRun(experiment, run_id='HD_b2777a12-2b03-4ce8-b460-7d5c097c2696')

For best results with Bayesian Sampling we recommend using a maximum number of runs greater than or equal to 20 times the number of hyperparameters being tuned. Recommendend value:40.


In [20]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [6]:
run.wait_for_completion()

{'runId': 'HD_b2777a12-2b03-4ce8-b460-7d5c097c2696',
 'target': 'auto-ml',
 'status': 'Completed',
 'startTimeUtc': '2022-03-13T17:06:53.128145Z',
 'endTimeUtc': '2022-03-13T17:30:08.991019Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name": "Accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '77989378-9273-42ad-bb13-e9bd70dbf05d',
  'user_agent': 'python/3.6.9 (Linux-5.4.0-1068-azure-x86_64-with-debian-buster-sid) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.38.0',
  'space_size': 'infinite_space_size',
  'score': '0.7631578947368421',
  'best_child_run_id': 'HD_b2777a12-2b03-4ce8-b460-7d5c097c2696_19',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://analyticsamlws1068325849.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_b2777a12-2b03-4ce8-b

 # Get Best Model

In [7]:
best_hd_run = run.get_best_run_by_primary_metric()
best_hd_run.get_details()

{'runId': 'HD_b2777a12-2b03-4ce8-b460-7d5c097c2696_19',
 'target': 'auto-ml',
 'status': 'Completed',
 'startTimeUtc': '2022-03-13T17:28:25.243955Z',
 'endTimeUtc': '2022-03-13T17:28:28.934272Z',
 'services': {},
 'properties': {'_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': '77989378-9273-42ad-bb13-e9bd70dbf05d',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [{'dataset': {'id': '101afb4f-c505-452c-8e1e-6fd1c02daf5f'}, 'consumptionDetails': {'type': 'Reference'}}],
 'outputDatasets': [],
 'runDefinition': {'script': 'train.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': ['--input-data',
   'heart',
   '--penalty',
   '7.0766762260864',
   '--kernel',
   'poly'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'auto-ml',
  'dataReferences': {},
  'data': {},
  'outputData': {},
  'datacaches': [],
  'jobName': None

In [8]:
from azureml.core import Model
best_model = best_hd_run.register_model("heart-hyperdrive", model_path="outputs/model.pkl")
best_model

Model(workspace=Workspace.create(name='analytics-aml-ws', subscription_id='67021087-2c78-4338-bc3d-dd21237069f1', resource_group='analytics-mlops-rg'), name=heart-hyperdrive, id=heart-hyperdrive:6, version=6, tags={}, properties={})

 # Inference Config

In [9]:
import os
source_dir_name = "source_dir_hd"
source_dir = f"./{source_dir_name}"
os.makedirs(source_dir, exist_ok=True)

In [10]:
from azureml.core import Environment
from azureml.core.model import InferenceConfig

env = Environment.from_conda_specification(name='sklearn-env', file_path='./source_dir_hd/conda_env_v_1_0_0.yml')

env.inferencing_stack_version='latest'
inference_config = InferenceConfig(
    environment=env,
    source_directory="./source_dir_hd",
    entry_script="score.py",
)

 # Deploy 

In [11]:
from azureml.core.webservice import AciWebservice, Webservice
deployment_config = AciWebservice.deploy_configuration(
    cpu_cores = 1,
    memory_gb = 1,
    auth_enabled=True,
    enable_app_insights=True)

In [13]:
service = AciWebservice(ws, "heart-predict-hd")

In [193]:
service = Model.deploy(
    ws,
    "heart-predict-hd",
    [best_model],
    inference_config,
    deployment_config,
    overwrite=True,
)
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-03-13 20:24:50+00:00 Creating Container Registry if not exists.
2022-03-13 20:24:50+00:00 Registering the environment.
2022-03-13 20:24:51+00:00 Use the existing image.
2022-03-13 20:24:51+00:00 Generating deployment configuration.
2022-03-13 20:24:52+00:00 Submitting deployment to compute.
2022-03-13 20:24:57+00:00 Checking the status of deployment heart-predict-hd..
2022-03-13 20:32:54+00:00 Checking the status of inference endpoint heart-predict-hd.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [14]:
logs = service.get_logs()
logs

'2022-03-13T20:32:43,103142900+00:00 - rsyslog/run \n2022-03-13T20:32:43,105010900+00:00 - gunicorn/run \n\nPATH environment variable: /azureml-envs/azureml_4943656157e05766fedf432d8be562a4/bin:/opt/miniconda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin\nPYTHONPATH environment variable: \n\nPip Dependencies\n---------------\n2022-03-13T20:32:43,112617300+00:00 - iot-server/run \n2022-03-13T20:32:43,119914900+00:00 - nginx/run \nrsyslogd: /azureml-envs/azureml_4943656157e05766fedf432d8be562a4/lib/libuuid.so.1: no version information available (required by rsyslogd)\nEdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...\n2022-03-13T20:32:43,603025900+00:00 - iot-server/finish 1 0\n2022-03-13T20:32:43,605050300+00:00 - Exit code 1 is normal. Not restarting iot-server.\nadal==1.2.7\napplicationinsights==0.11.10\nargcomplete==1.12.3\nazure-common==1.1.28\nazure-core==1.23.0\nazure-graphrbac==0.61.1\nazure-identity==1.7.0\nazure-mgmt-authorization==0.

 # Use Model

In [15]:
import json
data = {
  "data": [
    {
      "age": 20,
      "sex": 1,
      "cp": 0,
      "trtbps": 0,
      "chol": 0,
      "fbs": 0,
      "restecg": 0,
      "thalachh": 0,
      "exng": 0,
      "oldpeak": 0,
      "slp": 0,
      "caa": 0,
      "thall": 0
    }
  ],
  "method": "predict"
}
input_data = json.dumps(data)

In [16]:
response = service.run(input_data)
response

'{"result": [0]}'

In [17]:
import requests

scoring_uri = service.scoring_uri
key = service.get_keys()[0]
headers = {"Content-Type": "application/json",
"Authorization": f"Bearer {key}"}
response = requests.post(scoring_uri, input_data, headers=headers)
print(response)
print(response.json())

<Response [200]>
{"result": [0]}


 # Cleanup
 

In [21]:
from azureml.exceptions import ComputeTargetException
from azureml.core import ComputeTarget
try:
    compute = ComputeTarget(ws, amlcompute_cluster_name)
    compute.delete()
    print("Deleted cluster")
except ComputeTargetException:
    print("Cluster allready deleted")

Cluster allready deleted


In [18]:
from azureml.core.webservice import Webservice
Webservice.delete(service)